<a href="https://colab.research.google.com/github/cjstntjd/project_study/blob/master/video_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive',force_remount = True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import os
import numpy as np
import cv2

def save_image(path, success, is_resize, end_frame):
    
    cnt=0
    while success:
        successs,image= vidcap.read()
        if is_resize==1:
            image2= cv2.resize(image,(800,400))
        else:
            image2=image
        
        file_path = os.path.join(path, "frame%.5d.jpg" % cnt )
        cv2.imwrite(file_path, image2)
        cnt+=1
        
        print("%d image done " %cnt)
        
        if cnt== end_frame:
            break

end_frame = 400

print("save resize image")
vidcap = cv2.VideoCapture('/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/mov1.mp4')
success, image = vidcap.read()
save_image('/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/jpg', success, 1, end_frame)

print("save original image")
vidcap = cv2.VideoCapture('/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/mov1.mp4')
success, image = vidcap.read()
save_image('/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/jpg_ori', success, 0, end_frame)
#여기서부터 sub

image_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/jpg'
target_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/sub'
img_list = os.listdir(image_path)

cnt=0
is_save=1

for name in img_list:
    
    if cnt==0:
        prev_name = name
        
    else:
        prev_image_path = os.path.join(image_path, prev_name)
        cur_image_path = os.path.join(image_path, name)
        
        img_prev = cv2.imread(prev_image_path, cv2.IMREAD_GRAYSCALE)
        img_cur = cv2.imread(cur_image_path, cv2.IMREAD_GRAYSCALE)
        img_sub = cv2.subtract(img_prev, img_cur)
        
        prev_name = name
        
        if is_save==1:
            save_name = str("frame_sub%.5d.jpg" % cnt)
            save_path = os.path.join(target_path,save_name)
            cv2.imwrite(save_path, img_sub)
        
    cnt+=1

print("substract image process is complete")
#/////////////
img_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/sub'
target_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/sub_stack'
img_list = os.listdir(img_path)

img_H = 400
img_W = 800

window_size = 5

sub_tile_stack = np.zeros((img_H,img_W,window_size), dtype='f')
sub_tile_c = np.zeros((img_H,img_W), dtype='f')
sub_tile = np.zeros((img_H,img_W), dtype='f')

cnt = 0
stack_cnt = 0
is_save = 1

for name in img_list:

    cur_img_path  = os.path.join(img_path,name)
    
    
    sub_tile = cv2.imread(cur_img_path, cv2.IMREAD_GRAYSCALE)

    if stack_cnt >= window_size:
        sub_tile_c = sub_tile_c + sub_tile - sub_tile_stack[:,:,stack_cnt%window_size]
        sub_tile_stack[:,:,stack_cnt%window_size] = sub_tile
    else:
        sub_tile_stack[:,:,stack_cnt%window_size] = sub_tile
        sub_tile_c = sub_tile_c + sub_tile
 

    if is_save == 1:
        save_name = str("frame_sub%.5d.jpg" % cnt)
        save_path = os.path.join(target_path,save_name)
        cv2.imwrite(save_path, sub_tile_c) 

    cnt = cnt+1
    stack_cnt = stack_cnt+1
    
print('Stack images done')
#//////////////////////////////////////////////block
img_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/sub_stack'
target_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/sub_block'
img_list = os.listdir(img_path)

img_H = 400
img_W = 800

step_size = 10
step_H = round(img_H/step_size)
step_W = round(img_W/step_size)

print(step_H, step_W)
sub_tile = np.zeros((img_H,img_W))

cnt = 0
is_save = 1

thr = 40;

for name in img_list:

    cur_img_path  = os.path.join(img_path,name)

    # load two concecutive images
    img_cur = cv2.imread(cur_img_path, cv2.IMREAD_GRAYSCALE)
   
    for i in range(step_H):
        for j in range(step_W):
            img_crop=img_cur[i*step_size:(i+1)*step_size,j*step_size:(j+1)*step_size]
            crop_sum = np.sum(img_crop)
            sub_tile[i*step_size:(i+1)*step_size,j*step_size:(j+1)*step_size] = crop_sum/step_size/step_size*2

            if crop_sum/step_size/step_size*2 < thr:
                sub_tile[i*step_size:(i+1)*step_size,j*step_size:(j+1)*step_size] = 0
            

    if is_save == 1:
        save_name = str("frame_sub%.5d.jpg" % cnt)
        save_path = os.path.join(target_path,save_name)
        cv2.imwrite(save_path, sub_tile) 

    cnt = cnt+1
    
print('Block image done')
#///////////////////////
img_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/jpg_ori'
img_path_sub = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/sub_block'
target_path = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/highlight'

img_list = os.listdir(img_path)
img_list_sub = os.listdir(img_path_sub)


cnt = 0
is_save = 1

temp = np.zeros((360,640,3), dtype='f')
for i in range(len(img_list)):

    print('%s processing...' %i)
    if i == 0:
        print('skip first frame')
    else:
        img_file_path  = os.path.join(img_path,img_list[i])
        img_sub_file_path  = os.path.join(img_path_sub,img_list_sub[i-1])

        img = cv2.imread(img_file_path)
        img = cv2.resize(img, (640,360))
        img_sub = cv2.imread(img_sub_file_path, cv2.IMREAD_GRAYSCALE)
        img_sub = cv2.resize(img_sub, (640,360))
        
        
        temp[:,:,0] = img[:,:,0]
        temp[:,:,1] = img[:,:,1]
        temp[:,:,2] = img[:,:,2]        
        temp[:,:,2] = temp[:,:,2] + img_sub
    
        save_name = str("frame_sub%.5d.jpg" % cnt)
        save_path = os.path.join(target_path,save_name)
        cv2.imwrite(save_path, temp) 
        
        cnt = cnt+1

print('highlighting done')
#////////////////////////////////
image_folder = '/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/Frames/highlight'
video_name = 'car_move.mp4'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 22.5*33/31, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

print('finish')

save resize image
1 image done 
2 image done 
3 image done 
4 image done 
5 image done 
6 image done 
7 image done 
8 image done 
9 image done 
10 image done 
11 image done 
12 image done 
13 image done 
14 image done 
15 image done 
16 image done 
17 image done 
18 image done 
19 image done 
20 image done 
21 image done 
22 image done 
23 image done 
24 image done 
25 image done 
26 image done 
27 image done 
28 image done 
29 image done 
30 image done 
31 image done 
32 image done 
33 image done 
34 image done 
35 image done 
36 image done 
37 image done 
38 image done 
39 image done 
40 image done 
41 image done 
42 image done 
43 image done 
44 image done 
45 image done 
46 image done 
47 image done 
48 image done 
49 image done 
50 image done 
51 image done 
52 image done 
53 image done 
54 image done 
55 image done 
56 image done 
57 image done 
58 image done 
59 image done 
60 image done 
61 image done 
62 image done 
63 image done 
64 image done 
65 image done 
66 image done 
6

In [0]:
cd /gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video/

/gdrive/My Drive/스터디/개인과제/과속방지_이미지/car video
